In [30]:
import os
import pandas as pd 
import numpy as np

In [31]:
def count_performance_rows():
    PROCESSED_DIR = "/home/seb/Documents/prepayment/processed/"
    foreclosure_counts={}
    # read data from files
    # NOTE: Opening a file handler to read rather than Pandas.
    # Reason : We want to re
    with open(os.path.join(PROCESSED_DIR,"Performance.csv"),"r") as f:
        for i,line in enumerate(f):
            if i==0:
                continue
            loan_id,date=line.split(",")
            loan_id=int(loan_id)
            if loan_id not in foreclosure_counts:
                foreclosure_counts[loan_id]= {
                    "foreclosure_status": False,
                    "performance_count": 0
                } # initialize dict

            foreclosure_counts[loan_id]["performance_count"]+=1
            if len(date.strip())>0:
                foreclosure_counts[loan_id]["foreclosure_status"]=True
    return foreclosure_counts

In [32]:
def get_summary_of_performance(loan_id,key,foreclosure_count_dictionary):
    #The get method on dictionaries returns a default value if a key isn’t found.
    summary_value=foreclosure_count_dictionary.get(loan_id,{ 
        "foreclosure_status": False,
        "performance_count": 0
        })
    return summary_value[key]

In [33]:
def data_transform(acquisition,counts):
    MINIMUM_TRACKING_QUARTERS=2
    # add "foreclosure_status" column in acquisition dataframe by getting the values from the counts dictionary
    acquisition["foreclosure_status"]=acquisition["id"].apply(lambda x: get_summary_of_performance(x,"foreclosure_status",counts))

    # add "performance_count" column in acquisition dataframe by getting the values from the counts dictionary.
    acquisition["performance_count"]=acquisition["id"].apply(lambda x: get_summary_of_performance(x,"performance_count",counts))

    # convert date values - "first_payment_date" and "origination_date"
    for date in ["origination"]:
        cols="{}_date".format(date)
        acquisition["{}_month".format(date)]= pd.to_numeric(acquisition[cols].str.split('/').str.get(0))
        acquisition["{}_year".format(date)] = pd.to_numeric(acquisition[cols].str.split('/').str.get(1))

    acquisition = acquisition.fillna(-1)
    acquisition = acquisition[acquisition["performance_count"] > MINIMUM_TRACKING_QUARTERS]
    return acquisition

In [34]:
def read():
    PROCESSED_DIR = "/home/seb/Documents/prepayment/processed/"
    acquisition = pd.read_csv(os.path.join(PROCESSED_DIR, "Acquisition.csv"))
    return acquisition 

In [35]:
def write(acquisition):
    PROCESSED_DIR = "/home/seb/Documents/prepayment/processed/"
    acquisition.to_csv(os.path.join(PROCESSED_DIR, "train.csv"),index=False)

In [36]:
acquisition=read()
counts=count_performance_rows()
acquisition=data_transform(acquisition,counts)
write(acquisition)

In [37]:
file = '/home/seb/Documents/prepayment/processed/train.csv'
df=pd.read_csv(file)
print(df)

                   id  interest_rate origination_date  ltv   dti  \
0        100000506220          4.750          02/2014   80  40.0   
1        100006392566          3.500          01/2014   68  35.0   
2        100007679737          5.000          02/2014   80  31.0   
3        100008410787          4.375          12/2013   80  27.0   
4        100009360302          5.250          01/2014   80  38.0   
5        100013156268          4.625          01/2014   78  33.0   
6        100014615428          4.875          02/2014   80  44.0   
7        100018134297          4.375          12/2013   80  34.0   
8        100018812676          4.375          02/2014   49  18.0   
9        100023715413          5.000          01/2014   95  45.0   
10       100024428873          4.500          11/2013   62  34.0   
11       100029809089          4.500          01/2014   95  35.0   
12       100032422945          3.875          01/2014   79  42.0   
13       100033144008          5.125          01

In [46]:
default_list = df["foreclosure_status"].tolist()

In [47]:
default_count= 0
for i in default_list:
    if i is True:
        default_count+= 1
print(default_count)

4229


In [48]:
4229 / 3780262

0.0011187055288760408